In [8]:
# Import necessary libraries
import subprocess
import time

# Set the video URL, output file names, and maximum runtime
url = 'https://www.youtube.com/watch?v=ydYDqZQpim8'
output_file = 'video.mp4'
image_output_file = 'image%d.jpg'
max_runtime = '00:01:00'
parts = max_runtime.split(':')
max_runtime_seconds = int(parts[0]) * 3600 + int(parts[1]) * 60 + int(parts[2])

# Construct the command to download the video
command = ['youtube-dl', '-o', output_file, '--format', 'mp4', url,
           '|', 'ffmpeg', '-i', '-', '-t', max_runtime, output_file]

# Start the subprocess to download the video
process = subprocess.Popen(command, stdout=subprocess.PIPE)

# Set the interval for capturing images (in seconds)
interval = 10

# Set the starting time and image number
start_time = time.time()
image_number = 1

# Loop until the runtime has expired
while time.time() - start_time < max_runtime_seconds:

    # Capture an image from the video
    command = ['ffmpeg', '-i', output_file, '-ss',
               str(time.time() - start_time), image_output_file % image_number]
    subprocess.call(command)

    # Increment the image number
    image_number += 1

    # Wait for the next interval
    time.sleep(interval)
    



KeyboardInterrupt: 

In [5]:
print(start_time)

1670642706.6756754
